In [59]:
import sys
code_path = '../../network_bio_toolkit'
sys.path.append(code_path)

import importlib
import Heat
importlib.reload(Heat)

import create_graph
importlib.reload(create_graph)

from gprofiler import GProfiler
import networkx as nx
import pandas as pd
import community
import heat_and_cluster

import visJS2jupyter.visualizations as visualizations
importlib.reload(visualizations)

import visJS2jupyter.visJS_module as visJS_module
importlib.reload(visJS_module)

import matplotlib.pyplot as plt
import numpy as np

In [60]:
symbol = 'symbol'
entrez = 'entrez'

human = 'human'
mouse = 'mouse'

heat = Heat.Heat(gene_type = symbol, species = mouse)

In [61]:
filename = '../../background_networks/10090.protein.links.v10.5.txt'
heat.load_STRING_links(filename, confidence_filter = 700)

print('\nNumber of interactions: ' + str(len(list(heat.DG_universe.edges()))))

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-13911...done.
Finished.
110 input query terms found dup hits:
	[('ENSMUSP00000087485', 2), ('ENSMUSP00000000574', 2), ('ENSMUSP00000068487', 2), ('ENSMUSP000001132
403 input query terms found no hit:
	['ENSMUSP00000044573', 'ENSMUSP00000137380', 'ENSMUSP00000077813', 'ENSMUSP00000107421', 'ENSMUSP000
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Number of interactions: 369295


In [62]:
# load DEG file
DEG_filename = '../../DEG_databases/DE_CoeffspaceFlight - groundControl_glds48_20180312.csv'  
heat.create_DEG_list(DEG_filename, p_value_filter = 0.02, sep = ',')

print('Number of DEG\'s: ' + str(len(heat.DEG_list)))

Number of DEG's: 45


../../network_bio_toolkit\create_graph.py:277: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  df = pd.DataFrame.from_csv(filename, sep=sep)


In [63]:
saved_DG_universe = heat.DG_universe
heat.DG_universe = saved_DG_universe

In [64]:
heat.normalized_adj_matrix()

In [65]:
saved_Wprime = heat.Wprime
heat.Wprime = saved_Wprime

In [57]:
heat.draw_heat_prop(num_nodes = 500,
#                  random_walk = False,
                  edge_width = 2,
                  edge_smooth_enabled = True,
                  edge_smooth_type = 'bezier',
                  node_size_multiplier = 5,
                  hover = False,
                  hover_connected_edges = False,
                  largest_connected_component = True,
                  physics_enabled = True,
                  node_font_size = 40,
                  graph_id = 1,
                  node_shadow_x = 6)

In [58]:
heat.draw_clustering(rad_positions = False,
                k = None,
                largest_connected_component = True,
                num_top_genes = 500,
                cluster_size_cut_off = 5,
                remove_stray_nodes = True,
                node_spacing = 700,
                node_size_multiplier = 10,
                physics_enabled = True,
                node_font_size = 40,
                graph_id = 2,
                edge_width = 2,
                edge_smooth_enabled = True,
                edge_smooth_type = 'bezier',
                hover = False,
                hover_connected_edges = False
               )

In [66]:
heat.draw_clustering(num_top_genes = 500,
                cluster_size_cut_off = 5,
                remove_stray_nodes = True,
                r = 0.4,
                node_spacing = 600,
                node_size_multiplier = 4,
                physics_enabled = False,
                node_font_size = 15,
                graph_id = 3,
                node_shadow_x = 6,
                largest_connected_component = True
               )

TypeError: visjs_network() got an unexpected keyword argument 'plus'

In [ ]:
def draw_clustering(DG_universe, seed_nodes,
                    annotation = False,
                    rad_positions = True,
                    Wprime = None,
                    k = None,
                    largest_connected_component = False,
                    alpha = 0.5,
                    num_its = 20,
                    num_top_genes = 200,
                    cluster_size_cut_off = 0,
                    remove_stray_nodes = False,
                    r = 2.0,
                    x_offset = 2,
                    y_offset = 2,
                    node_spacing = 500,
                    node_size_multiplier = 10,
                    physics_enabled = False,
                    node_font_size = 40,
                    graph_id = 3,
                    **kwargs
                   ):
    
    # find hottest genes
    if Wprime is None:
        Wprime = visualizations.normalized_adj_matrix(DG_universe)
    
    Fnew = visualizations.network_propagation(nx.Graph(DG_universe), Wprime, seed_nodes)
    top_genes = Fnew.sort_values(ascending=False)[0:num_top_genes].index
    G_top_genes = nx.Graph(DG_universe).subgraph(top_genes) # casting to Graph to match heat prop
    
    # keep only the largest connected component
    G_top_genes = nx.Graph(G_top_genes)
    if largest_connected_component:
        G_top_genes = max(nx.connected_component_subgraphs(G_top_genes), key=len)

    # cluster hottest genes
    node_to_cluster = community.best_partition(G_top_genes)

    nodes = G_top_genes.nodes()
    edges = G_top_genes.edges()

    # color based on cluster
    node_to_color = assign_colors_to_clusters(node_to_cluster, cluster_size_cut_off)

    # remove stray nodes
    if remove_stray_nodes == True:
        not_grey_list = [n for n in nodes if node_to_color[n] != 'grey']
        node_to_cluster = {n:node_to_cluster[n] for n in nodes if n in not_grey_list}

        G_top_genes = nx.subgraph(G_top_genes, not_grey_list)
        nodes = G_top_genes.nodes()
        edges = G_top_genes.edges()
        
    # annotation
    if annotation == True:
        cluster_to_annotation = get_annotations(node_to_cluster)

    # position based on cluster
    if k is None:
        pos = nx.spring_layout(G_top_genes)
    else:
        pos = nx.spring_layout(G_top_genes,k=k)
    
    if rad_positions == True:
        bias_position_by_partition(pos, node_to_cluster, r = r, x_offset = x_offset, y_offset = y_offset); # modifies pos in place

    # set the title of each node
    nx.set_node_attributes(G_top_genes, 'cluster', node_to_cluster)
    node_titles = [str(node[0]) + '<br/>cluster = ' + str(node[1]['cluster'])
           for node in G_top_genes.nodes(data=True)]
    node_titles = dict(zip(nodes, node_titles))

    # label only seed nodes
    node_labels = {n:str(n) if n in seed_nodes else '' for n in G_top_genes.nodes()}

    # change shape of seed nodes
    node_shape = {n:'triangle' if n in seed_nodes else 'dot' for n in G_top_genes.nodes()}

    # create the nodes_dict with all relevant fields
    nodes_dict = [{'id':str(n),
           'color':node_to_color[n],
           'node_shape':node_shape[n],
           'node_label':node_labels[n],
           'title':node_titles[n],
           'x':pos[n][0]*node_spacing,
           'y':pos[n][1]*node_spacing} for n in nodes]

    # map nodes to indices for source/target in edges
    node_map = dict(zip(nodes,range(len(nodes))))

    # create the edges_dict with all relevant fields
    edges_dict = [{'source':node_map[edges[i][0]],
           'target':node_map[edges[i][1]],
           'color':'grey'} for i in range(len(edges))]

    return visJS_module.visjs_network(nodes_dict,edges_dict,
                                      node_label_field = 'node_label',
                                      node_size_multiplier = node_size_multiplier,
                                      physics_enabled = physics_enabled,
                                      node_font_size = node_font_size,
                                      graph_id = graph_id,
                                      **kwargs)

In [21]:
def get_annotations(node_to_cluster, top = None):
    
    cluster_to_node = heat_and_cluster.invert_dict(node_to_cluster)
    counter = 0
    
    if top == None:
        top = len(cluster_to_node) - 1
    
    gp = GProfiler('cluster_annotator', want_header = True)
    
    cluster_to_annotation = {}
    for cluster_id, group in cluster_to_node.items():
        
        print('Annotating cluster ' + str(cluster_id) + ' of ' + str(top) + '...')
        
        df = pd.DataFrame(gp.gprofile(group, organism = 'mmusculus'))
        df.columns = df.loc[0]
        df.drop(0)
        annotation = df['t name'][1]
        
        cluster_to_annotation[cluster_id] = annotation
        
        if counter == top:
            print('Done!')
            return cluster_to_annotation
        counter = counter + 1

In [179]:
gp.gprofile?


In [178]:
gp = GProfiler('cluster_annotator', want_header = True)
df = pd.DataFrame(gp.gprofile(heat.DG_universe.nodes()[0:10], organism = 'mmusculus'))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,#,signf,p-value,T,Q,Q&T,Q&T/Q,Q&T/T,term ID,t type,t group,t name,t depth,Q&T list
1,1,True,7.21e-07,98,8,5,0.625,0.051,KEGG:04713,keg,1,Circadian entrainment,1,"GNAI3,KCNJ3,GNGT2,RASD1,PRKCA"
2,1,True,1.55e-06,114,8,5,0.625,0.044,KEGG:04725,keg,1,Cholinergic synapse,1,"GNAI3,KCNJ3,GNGT2,PIK3R1,PRKCA"
3,1,True,6.31e-05,94,8,4,0.5,0.043,KEGG:05032,keg,1,Morphine addiction,1,"GNAI3,KCNJ3,GNGT2,PRKCA"
4,1,True,0.000137,114,8,4,0.5,0.035,KEGG:04724,keg,1,Glutamatergic synapse,1,"GNAI3,KCNJ3,GNGT2,PRKCA"
5,1,True,0.000239,131,8,4,0.5,0.031,KEGG:04926,keg,1,Relaxin signaling pathway,1,"GNAI3,GNGT2,PIK3R1,PRKCA"
6,1,True,0.000246,132,8,4,0.5,0.03,KEGG:04726,keg,1,Serotonergic synapse,1,"GNAI3,KCNJ3,GNGT2,PRKCA"
7,1,True,0.000254,133,8,4,0.5,0.03,KEGG:04728,keg,1,Dopaminergic synapse,1,"GNAI3,KCNJ3,GNGT2,PRKCA"
8,1,True,0.000378,147,8,4,0.5,0.027,KEGG:04723,keg,1,Retrograde endocannabinoid signaling,1,"GNAI3,KCNJ3,GNGT2,PRKCA"
9,1,True,0.000487,3,4,2,0.5,0.667,CORUM:1609,cor,1,"G protein complex (Gngt2, Kcnj3, Gnb1)",1,"KCNJ3,GNGT2"


In [71]:
annotation = True
node_spacing = 700
node_size_multiplier = 5
physics_enabled = False
node_font_size = 20
graph_id = 4

In [72]:
# run heat prop
seed_nodes = [n for n in heat.DEG_list if n in heat.DG_universe]
Fnew = visJS2jupyter.visualizations.network_propagation(nx.Graph(heat.DG_universe), heat.Wprime, seed_nodes)
top_genes = Fnew.sort_values(ascending=False)[0:500].index
G_top_genes = nx.Graph(heat.DG_universe).subgraph(top_genes) # casting to Graph to match heat prop

# keep only the largest connected component
G_top_genes = nx.Graph(G_top_genes)
G_top_genes = max(nx.connected_component_subgraphs(G_top_genes), key=len)

# cluster hottest genes
node_to_cluster = community.best_partition(G_top_genes) # no annotation nodes have a cluster id...

In [73]:
# annotation
cluster_to_annotation = get_annotations(node_to_cluster)
cluster_to_node = heat_and_cluster.invert_dict(node_to_cluster)

Annotating cluster 0 of 10...
Annotating cluster 1 of 10...
Annotating cluster 2 of 10...
Annotating cluster 3 of 10...
Annotating cluster 4 of 10...
Annotating cluster 5 of 10...
Annotating cluster 6 of 10...
Annotating cluster 7 of 10...
Annotating cluster 8 of 10...
Annotating cluster 9 of 10...
Annotating cluster 10 of 10...
Done!


In [75]:
edge_list = []
for cluster_id, node_group in cluster_to_node.items():
    anno = cluster_to_annotation[cluster_id]
    node_to_cluster[anno] = cluster_id
    anno_list = [anno]*len(node_group)
    new_edges = list(zip(anno_list, node_group))
    edge_list.extend(new_edges) 

In [76]:
  
# incorporate annotation into graph
G_top_genes.add_edges_from(edge_list)
nodes = G_top_genes.nodes()
edges = G_top_genes.edges()

# position based on cluster
pos = nx.spring_layout(G_top_genes)
heat_and_cluster.bias_position_by_partition(pos, node_to_cluster, r = 0.5); # modifies pos in place
    
# color based on cluster
node_to_color = heat_and_cluster.assign_colors_to_clusters(node_to_cluster, 5)

# color small clusters grey
not_grey_list = [n for n in nodes if node_to_color[n] != 'grey']
node_to_cluster = {n:node_to_cluster[n] for n in nodes if n in not_grey_list}
G_top_genes = nx.subgraph(G_top_genes, not_grey_list)

for k,v in cluster_to_annotation.items():
    node_to_color[v] = 'white'

In [77]:
# set the title of each node
nx.set_node_attributes(G_top_genes, 'cluster', node_to_cluster)
node_titles = [str(node[0]) + '<br/>cluster = ' + str(node[1]['cluster']) for node in G_top_genes.nodes(data=True)]
node_titles = dict(zip(nodes, node_titles))

In [114]:
# label only seed nodes
if annotation == False:
    node_labels = {n:str(n) if n in seed_nodes else '' for n in G_top_genes.nodes()}
else:
    node_labels = {n:'' for n in G_top_genes.nodes()}
    for k,v in cluster_to_annotation.items():
        node_labels[v] = v

In [174]:
# change shape of seed nodes
node_shape = {n:'triangle' if n in seed_nodes else 'dot' for n in G_top_genes.nodes()}

# set annotation nodes to be really small
node_to_size = {n:7 for n in G_top_genes.nodes()}
#for k,v in cluster_to_annotation.items():
#    node_to_size[v] = 1

# create the nodes_dict with all relevant fields
nodes_dict = [{'id':str(n),
       'color':node_to_color[n],
       'node_shape':node_shape[n],
       'node_label':node_labels[n],
       'node_size': node_to_size[n],
       'title':node_titles[n],
       'x':pos[n][0]*node_spacing,
       'y':pos[n][1]*node_spacing} for n in nodes]

# map nodes to indices for source/target in edges
node_map = dict(zip(nodes,range(len(nodes))))

# decide edge color
edge_list = [tuple(reversed(edge)) for edge in edge_list] # tuples are in the wrong direction for some reason
edge_to_color = {edge:'white' if edge in edge_list else 'grey' for edge in edges}

# move annotation nodes to front of list
nodes = list(reversed(nodes))
for edge in edge_list:
    edges.insert(0, edges.pop(edges.index(edge)))

In [176]:
importlib.reload(visJS_module)
# create the edges_dict with all relevant fields
edges_dict = [{'source':node_map[edges[i][0]],
       'target':node_map[edges[i][1]],
       'color':edge_to_color[edges[i]]} for i in range(len(edges))]

visJS_module.visjs_network(nodes_dict,edges_dict,
                                  node_label_field = 'node_label',
                                  node_size_field = 'node_size',
                                  physics_enabled = physics_enabled,
                                  node_font_size = 20,
                                  graph_id = graph_id)